In [1]:
import os
import sys 
import json
import glob
import random
import collections
import time
import re
import gc
import tracemalloc

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from scipy.spatial.distance import cdist

import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from sklearn.cluster import DBSCAN
from hdbscan import HDBSCAN
from torch.nn import functional as torch_functional
import torch.nn.functional as F

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

from tqdm import tqdm
import logging
from skimage.io import imread

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

from torch.autograd import Variable
from torch.optim import *
from scipy.io import loadmat
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, ConcatDataset, DataLoader, SubsetRandomSampler
import torchio as tio
from torch.utils.tensorboard import SummaryWriter

import matplotlib.cm as cm
import matplotlib.animation as animation
#import plotly.graph_objects as go
#import plotly.io as pio
#pio.renderers.default = "notebook"
import math
import datetime
import timeit

from scipy import io
import os

sys.path.append("..")
from utils.dataset_utils import *
from utils.classifier_utils import *

logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt="%Y-%m-%d %H:%M:%S")

In [2]:
ext = "mat"

dir_path = "../../RSNA-BTC-Datasets/train_"+ext
test_dir_path = "../../RSNA-BTC-Datasets/test_"+ext
ext_test_1_dir_path = "../../RSNA-BTC-Datasets/brats18_"+ext
ext_test_0_dir_path = "../../RSNA-BTC-Datasets/OpenNeuroDS000221_ss_"+ext
new_dir_path = "../../RSNA-BTC-Datasets/UPENN-GBM_"+ext
mri_types = ["FLAIR", "T1w"]#, "T1wCE", "T2w"]

dataset_0_mat = Dataset(dir_path, mri_types, list_classes=["0"], transform=None, ext=ext)
logging.info("Train/Val datasets size: {}".format(len(dataset_0_mat)))

dataset_1_mat = Dataset(dir_path, mri_types, list_classes=["1"], transform=None, ext=ext)
logging.info("Train/Val datasets size: {}".format(len(dataset_1_mat)))

dataset_test_mat = Dataset(test_dir_path, mri_types, list_classes=["unknown"], transform=None, ext=ext)
logging.info("Test datasets size: {}".format(len(dataset_test_mat)))

ext_test_dataset_1_mat = Dataset(ext_test_1_dir_path, mri_types, list_classes=["1"], transform=None, ext=ext)
logging.info("External Test dataset 1 size: {}".format(len(ext_test_dataset_1_mat)))

ext_test_dataset_0_mat = Dataset(ext_test_0_dir_path, mri_types, list_classes=["0"], transform=None, ext=ext)
logging.info("External Test dataset 0 size: {}".format(len(ext_test_dataset_0_mat)))

new_dataset_1_mat = Dataset(new_dir_path, mri_types, list_classes=["1"], transform=None, ext=ext)
logging.info("New dataset 1 size: {}".format(len(new_dataset_1_mat)))

new_dataset_0_mat = Dataset(new_dir_path, mri_types, list_classes=["0"], transform=None, ext=ext)
logging.info("New dataset 0 size: {}".format(len(new_dataset_0_mat)))

2022-07-08 17:41:32 - Train/Val datasets size: 270
2022-07-08 17:41:32 - Train/Val datasets size: 303
2022-07-08 17:41:32 - Test datasets size: 87
2022-07-08 17:41:32 - External Test dataset 1 size: 285
2022-07-08 17:41:32 - External Test dataset 0 size: 231
2022-07-08 17:41:32 - New dataset 1 size: 121
2022-07-08 17:41:32 - New dataset 0 size: 170


In [3]:
list_0 = []
list_1 = []
list_test = []
list_ext_test_1 = []
list_ext_test_0 = []
list_new_1 = []
list_new_0 = []
if True:
    print("Patients without tumor:")
    for i in range(len(dataset_0_mat)):
        #print(str(i)+" - "+dataset_0_flair_t2w.__getitem__(i)[0]+": "+str(dataset_0_flair_t2w.__getitem__(i)[4]))
        print(dataset_0_mat.__getid__(i))
        list_0.append(dataset_0_mat.__getid__(i))
        #print("---")
        #fig = plt.figure(dataset_flair_t2w.__getitem__(i)[0+idx]+": "+str(dataset_flair_t2w.__getitem__(i)[4+idx]))
        #plt.imshow(dataset_flair_t2w.__getitem__(i)[2+idx][128], cmap=cm.Greys_r, animated=True)

    #plt.show()
    print("Patients with tumor:")
    for i in range(len(dataset_1_mat)):
        #print(str(i)+" - "+dataset_0_flair_t2w.__getitem__(i)[0]+": "+str(dataset_0_flair_t2w.__getitem__(i)[4]))
        print(dataset_1_mat.__getid__(i))
        list_1.append(dataset_1_mat.__getid__(i))
        #print("---")
        
    print("Test patients:")
    for i in range(len(dataset_test_mat)):
        #print(str(i)+" - "+dataset_0_flair_t2w.__getitem__(i)[0]+": "+str(dataset_0_flair_t2w.__getitem__(i)[4]))
        print(dataset_test_mat.__getid__(i))
        list_test.append(dataset_test_mat.__getid__(i))
        #print("---")
        
    print("External Test 1 patients:")
    for i in range(len(ext_test_dataset_1_mat)):
        #print(str(i)+" - "+dataset_0_flair_t2w.__getitem__(i)[0]+": "+str(dataset_0_flair_t2w.__getitem__(i)[4]))
        print(ext_test_dataset_1_mat.__getid__(i))
        list_ext_test_1.append(ext_test_dataset_1_mat.__getid__(i))
        #print("---")
        
    print("External Test 0 patients:")
    for i in range(len(ext_test_dataset_0_mat)):
        #print(str(i)+" - "+dataset_0_flair_t2w.__getitem__(i)[0]+": "+str(dataset_0_flair_t2w.__getitem__(i)[4]))
        print(ext_test_dataset_0_mat.__getid__(i))
        list_ext_test_0.append(ext_test_dataset_0_mat.__getid__(i))
        #print("---")
        
    print("New 1 patients:")
    for i in range(len(new_dataset_1_mat)):
        #print(str(i)+" - "+dataset_0_flair_t2w.__getitem__(i)[0]+": "+str(dataset_0_flair_t2w.__getitem__(i)[4]))
        print(new_dataset_1_mat.__getid__(i))
        list_new_1.append(new_dataset_1_mat.__getid__(i))
        #print("---")
        
    print("New 0 patients:")
    for i in range(len(new_dataset_0_mat)):
        #print(str(i)+" - "+dataset_0_flair_t2w.__getitem__(i)[0]+": "+str(dataset_0_flair_t2w.__getitem__(i)[4]))
        print(new_dataset_0_mat.__getid__(i))
        list_new_0.append(new_dataset_0_mat.__getid__(i))
        #print("---")

Patients without tumor:
00009
00017
00018
00019
00021
00022
00024
00030
00032
00036
00044
00045
00049
00053
00061
00064
00072
00081
00084
00088
00090
00095
00097
00099
00102
00104
00110
00111
00112
00113
00116
00121
00122
00123
00124
00130
00133
00142
00149
00150
00151
00154
00157
00158
00162
00165
00167
00169
00170
00172
00176
00183
00184
00191
00192
00193
00194
00195
00201
00206
00209
00211
00214
00216
00217
00218
00219
00221
00227
00228
00231
00236
00237
00238
00239
00241
00242
00243
00247
00249
00251
00258
00259
00261
00262
00266
00267
00269
00274
00275
00280
00283
00286
00288
00289
00290
00297
00298
00300
00301
00308
00309
00310
00312
00314
00316
00318
00320
00324
00325
00327
00336
00339
00341
00343
00346
00347
00348
00349
00351
00353
00356
00373
00376
00377
00378
00379
00380
00382
00387
00388
00389
00390
00391
00392
00395
00397
00399
00401
00402
00405
00407
00410
00412
00414
00417
00418
00419
00421
00423
00430
00432
00433
00441
00444
00445
00446
00452
00454
00455
00459
00464
0046

In [4]:
def filter_slice(X, is_T1w=False):
    X_th = X.copy()
    X_th_f = X_th.flatten()
    X_top = X.copy()
    X_top_f = X_top.flatten()

    patches = plt.hist(X_th_f[X_th_f>np.min(X_th_f)+0.1], bins=100)
    n0 = [p for p in patches[0]]
    n1 = [p for p in patches[1]]
    
    m_hist = n1[np.argmax(n0)]
    
    if is_T1w:
        X_th_f = np.where((X_th_f < 0.1) | (X_th_f > m_hist), 0, 1)
    else:
        X_th_f[X_th_f<m_hist] = 0
        X_th_f[X_th_f>=m_hist] = 1
    
    X_th = X_th_f.reshape(192,-1)
    
    #X_th[X_th<m_hist] = 0
    #X_th[X_th>=m_hist] = 1
    #X_th = X_th.reshape(192,-1)
    
    if is_T1w:
        top_idxs = np.argsort(X_top_f)[:(np.count_nonzero(X==0)+np.count_nonzero(X_th)//2)]
        m_top = X_top_f[top_idxs[-1]]
        #top_20 = np.max(X_top_f)*4/5
        X_top_f = np.where((X_top_f < 0.1) | (X_top_f > m_top), 0, 1)
        #X_top_f[X_top_f<m_top] = 1
        #X_top_f[X_th_f>=m_top] = 0
    else:
        top_idxs = np.argsort(X_top_f)[-(np.count_nonzero(X_th)//3):]
        m_top = X_top_f[top_idxs[0]]
        #top_20 = np.max(X_top_f)*4/5
        X_top_f[X_top_f<m_top] = 0
        X_top_f[X_top_f>=m_top] = 1
    
    #top_idxs = np.argsort(X_top_f)[-(np.count_nonzero(X_th_f)//4):]
    #m_top = X_top_f[top_idxs[0]]
    #X_top_f[X_top_f<m_top] = 0
    #X_top_f[X_top_f>=m_top] = 1
    X_top = X_top_f.reshape(192,-1).astype(np.int16)
    
    plt.clf()
    plt.close()
    #del patches
    #del n0
    #del n1
    #del X_th
    #del X_th_f
    #del X_top_f
    #del top_idxs
    patches = None
    n0 = None
    n1 = None
    X_th = None
    X_th_f = None
    X_top_f = None
    top_idxs = None
    X = None
    m_hist = None
    m_top = None

    return X_top

def get_filtered_volume(dataset, patient_id, sub_dirs):
    X_dict = {}
    X_top_dict = {}
    for i in range(len(sub_dirs)):
        #X_i = []
        X_top_i = []
        X, target = get_patient_data(dataset, sub_dirs[i], patient_id)
        X = X - np.min(X)
        for j in range(192):
            #X, target = get_single_slice(dataset, dataset.sub_dirs[i], patient_id, j)
            #X = X - np.min(X)
            #X_i.append(X)
            is_T1w = sub_dirs[i] == "T1w" or sub_dirs[i] == "T1wCE"
            X_th = filter_slice(X[j], is_T1w=is_T1w)
            X_top_i.append(X_th)
            
        #X_i = np.stack(X,axis=0).T
        X_top_i = np.stack(X_top_i,axis=0)
        X_dict[sub_dirs[i]] = X
        X_top_dict[sub_dirs[i]] = X_top_i
        
        #del X
        #del X_top_i
        X = None
        X_top_i = None

        #print(f"{sub_dirs[i]}: {target}")
        
    X_int = np.where((X_top_dict["FLAIR"] == X_top_dict["T1w"]), X_top_dict["FLAIR"], 0)
    X_fil = np.where((X_int == 1), X_dict["FLAIR"], 0)
    #del X_dict
    #del X_int
    #del X_top_dict
    X_dict = None
    X_int = None
    X_top_dict = None
    #gc.collect()
    
    return X_fil, target

#https://nih.figshare.com/articles/dataset/MRI_dataset_supporting_Comparison_of_T1-Post_and_FLAIR-Post_MRI_for_identification_of_traumatic_meningeal_enhancement_in_traumatic_brain_injury_patients_/12386102
# https://openneuro.org/datasets/ds000221/versions/1.0.0

def save_volumes_as_mat(dataset, folder, target, patient_ids, sub_dirs):
    for patient_id in tqdm(patient_ids):
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/KLF/{target}/{patient_id}_KLF.mat"):
            img3d, t = get_filtered_volume(dataset, patient_id, sub_dirs)
            data = {'X': img3d, 'y': target}
            if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat"):
                os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat")
            if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/KLF"):
                os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat/KLF")  
            if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/KLF/{target}"):
                os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat/KLF/{target}")  
            if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/KLF/{target}/{patient_id}_KLF.mat"):
                io.savemat(f"../../RSNA-BTC-Datasets/{folder}_mat/KLF/{target}/{patient_id}_KLF.mat", data, do_compression=True)
            #del img3d
            #del data
            img3d = None
            data = None
            t = None
            
            #snapshot = tracemalloc.take_snapshot()
            #top_stats = snapshot.statistics('lineno')
            #for stat in top_stats:
            #    print(stat)
            

In [5]:
#tracemalloc.start()
save_volumes_as_mat(dataset_0_mat, "train", 0, list_0, ["FLAIR", "T1w"])
save_volumes_as_mat(dataset_1_mat, "train", 1, list_1, ["FLAIR", "T1w"])
save_volumes_as_mat(dataset_test_mat, "test", "unknown", list_test, ["FLAIR", "T1w"])
save_volumes_as_mat(ext_test_dataset_1_mat, "brats18", 1, list_ext_test_1, ["FLAIR", "T1w"])
save_volumes_as_mat(ext_test_dataset_0_mat, "OpenNeuroDS000221_ss", 0, list_ext_test_0, ["FLAIR", "T1w"])

100%|███████████████████████████████████████| 231/231 [3:06:03<00:00, 48.33s/it]


In [6]:
save_volumes_as_mat(new_dataset_1_mat, "UPENN-GBM", 1, list_new_1, ["FLAIR", "T1w"])
save_volumes_as_mat(new_dataset_0_mat, "UPENN-GBM", 0, list_new_0, ["FLAIR", "T1w"])

100%|███████████████████████████████████████| 170/170 [2:14:36<00:00, 47.51s/it]


In [ ]:
import sys
sys.path.append("..")
from utils.knowledge_filtering import *

In [ ]:
save_kb_volumes_as_mat(dataset_0_mat, "train", 0, list_0, ["FLAIR", "T1w"], "../../RSNA-BTC-Datasets")
save_kb_volumes_as_mat(dataset_1_mat, "train", 1, list_1, ["FLAIR", "T1w"], "../../RSNA-BTC-Datasets")
save_kb_volumes_as_mat(dataset_test_mat, "test", "unknown", list_test, ["FLAIR", "T1w"], "../../RSNA-BTC-Datasets")

In [6]:
#X_3d_1, _ = get_filtered_volume(dataset_1_mat, "00404", ["FLAIR", "T1w"])

In [7]:
%matplotlib notebook

frames = []
img = X_3d_1
fig = plt.figure("")
j = 0
while (j < len(img)):
    frames.append([plt.imshow(img[j], cmap=cm.Greys_r, animated=True)])
    j += 1

ani = animation.ArtistAnimation(fig, frames, interval=50, blit=True,
                                repeat_delay=1000)
plt.show()

NameError: name 'X_3d_1' is not defined

In [ ]:
%matplotlib inline